In [189]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import functional as F
import os
from pathlib import Path
from collections import defaultdict
import os
from argparse import Namespace
from joblib import dump, load
base_dir = Path('/Users/vinay/Projects/Recsys')
if not base_dir:
    base_dir = Path(os.getcwd())
from tqdm import tqdm

data_dir = base_dir/'data'/'archive'
store_dir = base_dir/'artifacts'
import wandb
from torch.utils.data import DataLoader as dl

papers used
- http://files.grouplens.org/papers/www10_sarwar.pdf
- https://arxiv.org/pdf/1907.06902.pdf

In [181]:
df = pd.read_csv(data_dir/'partial_rating.csv')

In [9]:
df['userId'].unique()

array([   1,    2,    3, ..., 6741, 6742, 6743])

When the entire data is partitioned,some of the movies have been removed. Thus we do not have some indices as clear from below.

In [12]:
df['movieId'].unique()

array([     2,     29,     32, ...,   5331,  96306, 100226])

In [182]:
movies = sorted(df['movieId'].unique())
users = sorted(df['userId'].unique())
movie_to_id = dict(zip(movies,range(len(movies))))
users_to_id = dict(zip(users,range(len(users))))

In [107]:

def make_user_movie_mat(df,user_sz=None,movie_sz=None):
    movies = sorted(df['movieId'].unique())
    users = sorted(df['userId'].unique())
    
    movie_to_id = dict(zip(movies,range(len(movies))))
    users_to_id = dict(zip(users,range(len(users))))
    if (user_sz != None):
        user_movie_mat = np.zeros((user_sz,movie_sz))
    else:
        user_movie_mat = np.zeros((len(users),len(movies)))
    for i in tqdm(range(len(df))):
        usr = df.iloc[i]['userId']
        mv = df.iloc[i]['movieId']
        rtg = df.iloc[i]['rating']
        row,col = users_to_id[usr],movie_to_id[mv]
        try:
            user_movie_mat[row][col] = rtg
        except:
            import pdb;pdb.set_trace()
    return user_movie_mat

In [46]:
#initialize a matrix of all 0's

make_user_movie_mat(df)


100%|███████████████████████████████| 1000390/1000390 [01:43<00:00, 9695.11it/s]


Above `user_movie_mat` is a matrix of size #users,#movies with each element being the corresponding rating. Unseen movies are left with 0.

In [41]:
dump(user_movie_mat,data_dir/'user_movie_mat.joblib')

['/Users/vinay/Projects/Recsys/data/archive/user_movie_mat.joblib']

In [42]:
user_movie_mat = load(data_dir/'user_movie_mat.joblib')

In [43]:
user_movie_mat

array([[-1. ,  3.5, -1. , ..., -1. , -1. , -1. ],
       [-1. , -1. ,  4. , ..., -1. , -1. , -1. ],
       [ 4. , -1. , -1. , ..., -1. , -1. , -1. ],
       ...,
       [ 4. , -1. , -1. , ..., -1. , -1. , -1. ],
       [-1. , -1. , -1. , ..., -1. , -1. , -1. ],
       [ 4. ,  3. ,  3. , ..., -1. , -1. , -1. ]])

In [214]:
df_trn = pd.read_csv(data_dir/'partial_random_trn.csv')
df_vld = pd.read_csv(data_dir/'partial_random_vld.csv')
df_tst = pd.read_csv(data_dir/'partial_random_tst.csv')

Here I'm merging the train and valid datasets into one. I will make multiple splits of this dataset to tune hyperparameters.

In [66]:
df_vld.index = range(len(df_trn),len(df_vld)+len(df_trn),1)

In [216]:
df_trn_vld = pd.concat([df_trn,df_vld])
df_trn_vld

,Unnamed: 0,user_id,rating,movie_id
0,0,1,4.0,541
1,1,1,4.0,2021
2,2,1,3.5,1222
3,3,1,4.0,318
4,4,1,4.0,6539
...,...,...,...,...
200018,200018,6743,3.5,2712
200019,200019,6743,4.0,5959
200020,200020,6743,2.5,786
200021,200021,6743,3.5,208


In [79]:
def get_data(data_dir,split:list,mode,data_type,save_to_disk=False,only_train_valid=False,data=None):
    #split = [0.6,0.5]
    #mode = 'random' or 'seq_aware'
    #save_to_disk = flag to specify if we also want to save the splitted data to disk to avoid future computation.
    # We are currently using 'random' split-> for each user, we take all the rows(movies he rated ) and randomly split
    #them between trn,vld,tst
    trn,vld,tst = defaultdict(list),defaultdict(list),defaultdict(list)
    if data_dir:
        import pdb;pdb.set_trace()
        df = pd.read_csv(data_dir/(data_type+'_'+'rating.csv'))
        df['date'] = pd.to_datetime(df['timestamp'])
        user_ids = set(df['userId'])
    else:
        df = data
        user_ids = set(df['user_id'])
        
    #Reorder rows
    perm = random.sample(range(len(df)),len(df))
    df = df.iloc[perm].reset_index()
    
    for user in tqdm(user_ids):
        
        #splitting-ids
        if not only_train_valid:
            tmp = df[df['userId'] == user]
            if split == 'seq_aware':
                tmp = tmp.sort_values(by='date')
            sz = len(tmp)
            t,v = int(split[0]*sz),int(split[0]*sz)+int((sz-int(split[0]*sz))*split[-1])
            trn_ids,vld_ids,tst_ids = slice(0,t),slice(t,v),slice(v,sz)

            trn['user_id'].extend(tmp[trn_ids]['userId'].tolist())
            trn['rating'].extend(tmp[trn_ids]['rating'].tolist())
            trn['movie_id'].extend(tmp[trn_ids]['movieId'].tolist())


            vld['user_id'].extend(tmp[vld_ids]['userId'].tolist())
            vld['rating'].extend(tmp[vld_ids]['rating'].tolist())
            vld['movie_id'].extend(tmp[vld_ids]['movieId'].tolist())

            tst['user_id'].extend(tmp[tst_ids]['userId'].tolist())
            tst['rating'].extend(tmp[tst_ids]['rating'].tolist())
            tst['movie_id'].extend(tmp[tst_ids]['movieId'].tolist())
        else:
            tmp = df[df['user_id'] == user]
            sz = len(tmp)
            t = int(split[0]*sz)
            trn_ids,vld_ids = slice(0,t),slice(t,sz)
            
            trn['user_id'].extend(tmp[trn_ids]['user_id'].tolist())
            trn['rating'].extend(tmp[trn_ids]['rating'].tolist())
            trn['movie_id'].extend(tmp[trn_ids]['movie_id'].tolist())


            vld['user_id'].extend(tmp[vld_ids]['user_id'].tolist())
            vld['rating'].extend(tmp[vld_ids]['rating'].tolist())
            vld['movie_id'].extend(tmp[vld_ids]['movie_id'].tolist())


            
    trn,vld,tst = pd.DataFrame(trn),pd.DataFrame(vld),pd.DataFrame(tst)
    if save_to_disk:
        trn.to_csv(data_dir/(data_type+'_'+mode+'_trn.csv'))
        vld.to_csv(data_dir/(data_type+'_'+mode+'_vld.csv'))
        tst.to_csv(data_dir/(data_type+'_'+mode+'_tst.csv'))
    return trn,vld,tst
        



Above function is ugly but esentially it just gives a random split of the combined data-frame with a specified ratio.

In [80]:
trn_split,vld_split,_ = get_data(None,[0.7],'random','',save_to_disk=False,only_train_valid=True,data=df_trn_vld)

100%|█████████████████████████████████████| 6743/6743 [00:04<00:00, 1368.21it/s]


In [98]:
trn_split.rename(columns={'user_id':'userId','movie_id':'movieId'},inplace=True)
vld_split.rename(columns={'user_id':'userId','movie_id':'movieId'},inplace=True)

In [134]:
mv_sz,user_sz = len(df['movieId'].unique()),len(df['userId'].unique())

trn_user_movie_mat = make_user_movie_mat(trn_split,user_sz,mv_sz)


100%|████████████████████████████████| 555449/555449 [00:52<00:00, 10492.36it/s]


In [166]:
#normalize the matrix
#trn_user_movie_mat = trn_user_movie_mat/(np.linalg.norm(trn_user_movie_mat,axis=-1))
trn_user_movie_mat.shape

(6743, 13950)

In [168]:
scaled_trn_user_movie_mat = trn_user_movie_mat/(np.linalg.norm(trn_user_movie_mat,axis=0)+1e-6)

In [170]:
scaled_trn_user_movie_mat.shape

(6743, 13950)

In [171]:
#collect all pair wise dot-products.
dot_product_mat = (trn_user_movie_mat.T)@trn_user_movie_mat

In [172]:
dot_product_mat[9][[0,9,8]]

array([ 3511.  , 10094.75,   280.5 ])

In [202]:
def get_rating(user_id,movie_id,train_df,similarity_mat,top=10):
    movies_seen = np.array(train_df[train_df['userId'] == user_id]['movieId'])
    corres_ratings = np.array(train_df[train_df['userId'] == user_id]['rating'])
    movie_ids = []
    movie_to_comp = movie_to_id[movie_id]
    for mv in movies_seen:
        movie_ids.append(movie_to_id[mv])
        
    similarity_vals = similarity_mat[movie_to_comp][movie_ids]
    if len(similarity_vals)<top:
        top = len(similarity_vals)
    
    try:
        top_indices = np.argsort(similarity_vals)[-top:]
        top_similar_vals= similarity_vals[top_indices]
        top_ratings = corres_ratings[top_indices]
    except:
         import pdb;pdb.set_trace()
        
    
    weighted_rating = sum(top_ratings*(top_similar_vals/(1e-6+sum(top_similar_vals))))
    return weighted_rating
    
    

In [177]:
r = get_rating(1,8507,trn_split,dot_product_mat,top=10)

In [178]:
r

3.4172846164217336

In [219]:

def cross_valid_best_params(df_trn,df_vld,df,top_list=[5,10,15,20,30],times=3,combine=True):
    out_dict = defaultdict(list)
    if combine:
        df_vld.index = range(len(df_trn),len(df_vld)+len(df_trn),1)
        df_trn_vld = pd.concat([df_trn,df_vld])
    else:
        df_trn_vld = df_trn
    for top_k in top_list:
        print(f"at the start of : {top_k}")
        # do it for 3 splits and average the results
        error = []
        for i in range(times):
            tst_error = []
            if combine:
                trn_split,vld_split,_ = get_data(None,[0.7],'random','',save_to_disk=False,only_train_valid=True,data=df_trn_vld)  
            else:
                trn_split,vld_split = df_trn,df_vld
                
            trn_split.rename(columns={'user_id':'userId','movie_id':'movieId'},inplace=True)
            vld_split.rename(columns={'user_id':'userId','movie_id':'movieId'},inplace=True)
            mv_sz,user_sz = len(df['movieId'].unique()),len(df['userId'].unique())
            trn_user_movie_mat = make_user_movie_mat(trn_split,user_sz,mv_sz)
            scaled_trn_user_movie_mat = trn_user_movie_mat/(np.linalg.norm(trn_user_movie_mat,axis=0)+1e-6)
            #collect all pair wise dot-products.
            dot_product_mat = (trn_user_movie_mat.T)@trn_user_movie_mat
            print(f"Inside validation loop")
            for i in tqdm(range(len(vld_split))):
                try:
                    usr = vld_split.iloc[i]['userId']
                    mv = vld_split.iloc[i]['movieId']
                    rtg = vld_split.iloc[i]['rating']
                except:
                    print(f"check the column names for the df")
                    import pdb;pdb.set_trace()
                pred_r = get_rating(usr,mv,trn_split,dot_product_mat,top=top_k)
                tst_error.append(abs(rtg-pred_r))
            error.append(np.mean(tst_error))
            print(f"top {top_k},iter {i} -> avg.abs error {error[-1]}")
        out_dict[top_k].append(np.mean(error))
    return out_dict
            


        

In [205]:
out_error = cross_valid_best_params(df_trn,df_vld,df)

at the start of : 5


100%|████████████████████████████████| 555449/555449 [00:53<00:00, 10318.44it/s]


Inside validation loop


100%|██████████████████████████████████| 242245/242245 [04:59<00:00, 808.86it/s]


top 5,iter 242244 -> avg.abs error 0.8779272686057658


...

top 30,iter 242244 -> avg.abs error 0.8180613293398472


100%|████████████████████████████████| 555449/555449 [00:54<00:00, 10266.73it/s]


Inside validation loop


100%|██████████████████████████████████| 242245/242245 [04:59<00:00, 809.44it/s]

top 30,iter 242244 -> avg.abs error 0.8266165541040442


In [207]:
dump(out_error,data_dir/'top_k.joblib')

['/Users/vinay/Projects/Recsys/data/archive/top_k.joblib']

In [222]:
out_error

defaultdict(list,
            {5: [0.8748709148995161],
             10: [0.8404949263267172],
             15: [0.8348021640050689],
             20: [0.8268270866438873],
             30: [0.8217495592890863]})

In [210]:
out_error2 = cross_valid_best_params(df_trn,df_vld,df,top_list=[35,40,45,50])

at the start of : 35


100%|█████████████████████████████████| 555449/555449 [1:34:51<00:00, 97.59it/s]


Inside validation loop


100%|████████████████████████████████| 242245/242245 [10:22:01<00:00,  6.49it/s]


top 35,iter 242244 -> avg.abs error 0.8341438371008657


...

top 50,iter 242244 -> avg.abs error 0.8219372724239943


100%|████████████████████████████████| 555449/555449 [00:53<00:00, 10312.30it/s]


Inside validation loop


100%|██████████████████████████████████| 242245/242245 [04:58<00:00, 811.76it/s]


top 50,iter 242244 -> avg.abs error 0.8294910598162443


In [224]:
store = {**out_error2,**out_error }
dump(store,data_dir/'top_k.joblib')

['/Users/vinay/Projects/Recsys/data/archive/top_k.joblib']

In [212]:
# So the number of neighbours is fixed at 30

In [220]:
test_mae = cross_valid_best_params(df_trn_vld,df_tst,df,top_list=[30],times=1,combine=False)

at the start of : 30


100%|████████████████████████████████| 797694/797694 [01:18<00:00, 10169.95it/s]


Inside validation loop


100%|██████████████████████████████████| 202696/202696 [05:37<00:00, 599.88it/s]


top 30,iter 202695 -> avg.abs error 0.7878955398684241


In [225]:
dump(test_mae,data_dir/'test_mae_baseline.joblib')

['/Users/vinay/Projects/Recsys/data/archive/test_mae_baseline.joblib']

In [226]:
test_2 = pd.read_csv('/Users/vinay/Projects/Recsys/artifacts/2_movielens_partial/inference_random_tst.csv')

In [230]:
np.mean(np.abs(test_2['y_true']-test_2['y_pred']))

0.6613349791993923